In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.multiclass import OneVsRestClassifier
from imblearn.over_sampling import RandomOverSampler

ModuleNotFoundError: No module named 'imblearn'

In [2]:
pd.set_option('display.max_columns', None)

# Using exploded data tabel

In [3]:

wine_df_1 = pd.read_csv(f'~/code/ArjanAngenent/VinoDine/raw_data/Cleaned_Full_100K_wines_exploded.csv')
wine_df_1

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100003,Cabernet Sauvignon,Red,Varietal/100%,CabernetSauvignon,Beef,12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
1,100003,Cabernet Sauvignon,Red,Varietal/100%,CabernetSauvignon,Lamb,12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,CabernetSauvignon,Poultry,12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,CabernetSauvignon,Beef,11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,Merlot,Beef,11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527951,200795,Rotleiten,Red,Assemblage/Blend,CabernetSauvignon,SpicyFood,13.5,Medium-bodied,High,AT,Austria,2093,Wien,62795,Zahel,http://www.zahel.at,"[2020, 2019, 2017, 2015, 2014, 2013, 2012, 201..."
527952,200795,Rotleiten,Red,Assemblage/Blend,CabernetSauvignon,MaturatedCheese,13.5,Medium-bodied,High,AT,Austria,2093,Wien,62795,Zahel,http://www.zahel.at,"[2020, 2019, 2017, 2015, 2014, 2013, 2012, 201..."
527953,200795,Rotleiten,Red,Assemblage/Blend,CabernetSauvignon,HardCheese,13.5,Medium-bodied,High,AT,Austria,2093,Wien,62795,Zahel,http://www.zahel.at,"[2020, 2019, 2017, 2015, 2014, 2013, 2012, 201..."
527954,200795,Rotleiten,Red,Assemblage/Blend,CabernetSauvignon,Poultry,13.5,Medium-bodied,High,AT,Austria,2093,Wien,62795,Zahel,http://www.zahel.at,"[2020, 2019, 2017, 2015, 2014, 2013, 2012, 201..."


In [4]:
wine_df_1['Harmonize'].nunique()

23

In [5]:
wine_df_1['Grapes'].nunique()

50

In [6]:
wine_df_1['Acidity'].unique()

array(['High', 'Medium', 'Low'], dtype=object)

## create base model

In [7]:
# features to select: Type, Elaborate, Grapes, ABV, Body, Acidity
# target: harmonize
y = wine_df_1[['Harmonize']]
X = wine_df_1[['Type', 'Elaborate', 'ABV', 'Body', 'Grapes', 'Acidity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
ohe_y = OneHotEncoder(sparse_output=False)

# Create Ordinal Enoder for Acidity with 3 categories ['High', 'Medium', 'Low']
ordinal_encoder = OrdinalEncoder(categories=[['High', 'Medium', 'Low']])

# Create StandardScaler for ABV(numerical feeature)
standard_scaler = StandardScaler()

# fit_transform encoders and scalers to X_train
X_train[ohe.get_feature_names_out()] = ohe.fit_transform(X_train[['Type','Elaborate','Body', 'Grapes']])

X_train['encoded_acidicity'] = ordinal_encoder.fit_transform(X_train[['Acidity']])

X_train['ABV'] = standard_scaler.fit_transform(X_train[['ABV']])

X_train.drop(columns=['Type','Elaborate','Body', 'Acidity', 'Grapes'], inplace=True)

#y_train = ohe_y.fit_transform(y_train)

# transform X_test with encoders and scalers
X_test[ohe.get_feature_names_out()] = ohe.transform(X_test[['Type','Elaborate','Body', 'Grapes']])

X_test['encoded_acidicity'] = ordinal_encoder.transform(X_test[['Acidity']])

X_test['ABV'] = standard_scaler.transform(X_test[['ABV']])

X_test.drop(columns=['Type','Elaborate','Body', 'Acidity', 'Grapes'], inplace=True)

#y_test = ohe_y.transform(y_test)

In [8]:
X_train.head()

,ABV,Type_Dessert,Type_Dessert/Port,Type_Red,Type_Rosé,Type_Sparkling,Type_White,Elaborate_Assemblage/Blend,Elaborate_Assemblage/Bordeaux Red Blend,Elaborate_Assemblage/Bourgogne Red Blend,Elaborate_Assemblage/Bourgogne White Blend,Elaborate_Assemblage/Cava Blend,Elaborate_Assemblage/Champagne Blend,Elaborate_Assemblage/Chianti Red Blend,Elaborate_Assemblage/Meritage Red Blend,Elaborate_Assemblage/Meritage White Blend,Elaborate_Assemblage/Port Blend,Elaborate_Assemblage/Portuguese Red Blend,Elaborate_Assemblage/Portuguese White Blend,Elaborate_Assemblage/Priorat Red Blend,Elaborate_Assemblage/Provence Rosé Blend,Elaborate_Assemblage/Rhône Red Blend,Elaborate_Assemblage/Rioja Red Blend,Elaborate_Assemblage/Soave White Blend,Elaborate_Assemblage/Tuscan Red Blend,Elaborate_Assemblage/Valpolicella Red Blend,Elaborate_Varietal/100%,Elaborate_Varietal/>75%,Body_Full-bodied,Body_Light-bodied,Body_Medium-bodied,Body_Very full-bodied,Body_Very light-bodied,Grapes_AlicanteBouschet,Grapes_Aragonez,Grapes_Barbera,Grapes_CabernetFranc,Grapes_CabernetSauvignon,Grapes_Carignan/Cariñena,Grapes_Carmenère,Grapes_Chardonnay,Grapes_CheninBlanc,Grapes_Cinsault,Grapes_Cortese,Grapes_Corvina,Grapes_GamayNoir,Grapes_Garganega,Grapes_Garnacha,Grapes_Gewürztraminer,Grapes_Glera/Prosecco,Grapes_Grenache,Grapes_GrenacheBlanc,Grapes_GrünerVeltliner,Grapes_Macabeo,Grapes_Malbec,Grapes_Merlot,Grapes_Molinara,Grapes_Montepulciano,Grapes_Mourvedre,Grapes_Nebbiolo,Grapes_PetitVerdot,Grapes_PetiteSirah,Grapes_PinotBlanc,Grapes_PinotGrigio,Grapes_PinotGris,Grapes_PinotMeunier,Grapes_PinotNoir,Grapes_Primitivo,Grapes_Riesling,Grapes_Rondinella,Grapes_Roussanne,Grapes_Sangiovese,Grapes_SauvignonBlanc,Grapes_Spätburgunder,Grapes_Syrah/Shiraz,Grapes_Sémillon,Grapes_Tempranillo,Grapes_TintaRoriz,Grapes_TourigaFranca,Grapes_TourigaNacional,Grapes_Trebbiano,Grapes_Viognier,Grapes_Zinfandel,encoded_acidicity
93128,-0.631335,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
421330,0.089598,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214775,0.089598,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211639,0.089598,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90833,0.450065,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
y_train.shape

(369569, 1)

In [10]:
model = LogisticRegression()
# Make it an Multilabel classifier
#multilabel_classifier = MultiOutputClassifier(model, n_jobs=-1)

# Fit the data to the Multilabel classifier
#multilabel_classifier = multilabel_classifier.fit(X_train, y_train)
model.fit(X_train, y_train)

/home/jameskubik/.pyenv/versions/3.10.6/envs/VinoDine/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jameskubik/.pyenv/versions/3.10.6/envs/VinoDine/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
model = LogisticRegression(max_iter=1000)
OVR = OneVsRestClassifier(model)
OVR.fit(X_train, y_train)

In [ ]:
# cross_val_score(estimator=multilabel_classifier,X=X_train,y=y_train, cv=5, n_jobs=-1)

In [ ]:
y_pred = OVR.predict(X_test)
y_pred

In [ ]:
# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
print(classification_report(y_test, y_pred))

# Using non-exploded data table

In [ ]:
save_file_name = "Cleaned_Full_100K_wines"
mlb_harm = MultiLabelBinarizer(sparse_output=False)
mlb_grape = MultiLabelBinarizer(sparse_output=False)

wine_df_2 = pd.read_csv(f'~/code/ArjanAngenent/VinoDine/raw_data/{save_file_name}.csv')

In [ ]:
mlb_harm = MultiLabelBinarizer(sparse_output=False)
mlb_grape = MultiLabelBinarizer(sparse_output=False)

y_2 = wine_df_2[['Harmonize']]
X_2 = wine_df_2[['Type', 'Elaborate', 'Grapes', 'ABV', 'Body', 'Acidity']]

In [ ]:
y_2_bin = mlb_harm.fit_transform(eval(element) for element in y_2.Harmonize)
y_2_bin

In [ ]:
X_2_bin = X_2.join(pd.DataFrame(
    mlb_grape.fit_transform(eval(element) for element in X_2.Grapes),
    index=X_2.index,
    columns=mlb_grape.classes_
    ))
X_2_bin.drop(columns='Grapes', inplace=True)
X_2_bin


In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2_bin, y_2_bin, random_state=42, test_size=0.3)

In [ ]:
X_train_2.shape, X_test_2.shape, y_train_2.shape, y_test_2.shape 

## Model definition

In [ ]:
# Define which columns need to be encoded
cat_cols = ['Type','Elaborate','Body','Acidity']
num_cols = ['ABV']
cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                        MinMaxScaler())
cat_num = make_pipeline(MinMaxScaler())

# Create preprocessor pipeline
preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

# Create binary classifier
binary_classifier = SGDClassifier(max_iter=500, random_state=42)

ova_classifier = OneVsRestClassifier(binary_classifier)

pipeline = make_pipeline(preprocessing, ova_classifier)

In [ ]:
pipeline.fit(X_train_2, y_train_2)

In [ ]:
y_pred_2 = ova_classifier.predict(preprocessing.transform(X_test_2))

In [ ]:
# Evaluate performance
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
print("Accuracy:", accuracy_2)

In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(y_test_2, y_pred_2))

# Reducing features and targets

In [ ]:
save_file_name = "Cleaned_Full_100K_wines"
mlb_harm = MultiLabelBinarizer(sparse_output=False)
mlb_grape = MultiLabelBinarizer(sparse_output=False)

wine_df_2 = pd.read_csv(f'~/code/ArjanAngenent/VinoDine/raw_data/{save_file_name}.csv')


# Binary encode grapes
wine_df_2_bin = wine_df_2.join(pd.DataFrame(
    mlb_grape.fit_transform(eval(element) for element in wine_df_2.Grapes),
    index=wine_df_2.index,
    columns=mlb_grape.classes_
    ))
wine_df_2_bin.drop(columns=['Grapes'], inplace=True)

# Create a list of the kind of grapes that are mentioned less then 2.000 times
grapes_list = wine_df_2_bin.iloc[:,16:].sum() # sum the number of times a grape is mentioned via column
grapes_to_drop = grapes_list[grapes_list<2_000].index.to_list() # create a list withe kind of grapes mentioned less then 2.000 times

wine_df_2_bin.drop(columns=grapes_to_drop, inplace=True) # drop columns with grapes not mentioned more then 2.000 times
wine_df_2_bin = wine_df_2_bin[wine_df_2_bin.iloc[:,16:].eq(1).any(axis=1)] # drop wines which are not represented by a grape anymore

# Binary encode Harmonize(kinds of food)
wine_df_2_bin = wine_df_2_bin.join(pd.DataFrame(
    mlb_harm.fit_transform(eval(element) for element in wine_df_2.Harmonize),
    index=wine_df_2.index,
    columns=mlb_harm.classes_
    ))
wine_df_2_bin.drop(columns=['Harmonize'], inplace=True)

# Create a list of the kind of grapes that are mentioned less then 2.000 times
harm_list = wine_df_2_bin.iloc[:,29:].sum() # sum the number of times a food is mentioned via column
harm_to_drop = harm_list[harm_list<=5_000].index.to_list() # create a list withe kind of food mentioned less then 50 times
wine_df_2_bin.drop(columns=harm_to_drop, inplace=True) # drop columns with food not mentioned more then 50 times
wine_df_2_bin = wine_df_2_bin[wine_df_2_bin.iloc[:,29:].eq(1).any(axis=1)] # drop wines which are not represented by a food anymore

#Drop addional columns not used for model
wine_df_2_bin_cleaned = wine_df_2_bin.drop(columns=['WineName', 'WineID','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])

In [ ]:
wine_df_2_bin_cleaned

In [ ]:
# Define which columns need to be encoded
cat_cols = ['Type','Elaborate','Body','Acidity']
# cat_cols = ['Type', 'Acidity','Elaborate']
num_cols = ['ABV']
cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                        MinMaxScaler())
cat_num = make_pipeline(MinMaxScaler())

# Create preprocessor pipeline
preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

# Create binary classifier
binary_classifier = SGDClassifier(max_iter=500, random_state=42)

ova_classifier = OneVsRestClassifier(binary_classifier)

pipeline = make_pipeline(preprocessing, ova_classifier)

In [ ]:
#X_3 = wine_df_2_bin_cleaned[['Type','Elaborate','Body','Acidity', 'ABV']]
X_3 = wine_df_2_bin_cleaned.iloc[:,:5]
y_3 = wine_df_2_bin_cleaned.iloc[:,19:]

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, random_state=42, test_size=0.3)

X_train_3.shape, X_test_3.shape, y_train_3.shape, y_test_3.shape 

In [ ]:
pipeline.fit(X_train_3, y_train_3)

y_pred_3 = ova_classifier.predict(preprocessing.transform(X_test_3))

# Evaluate performance
accuracy_3 = accuracy_score(y_test_3, y_pred_3)
print("Accuracy:", accuracy_3)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_3, y_pred_3))

In [ ]:
y_3.columns

# Aggregating Harmonization

In [ ]:
def combine_harmonize_classes(harmonize):
    if harmonize in ['Beef', 'Lamb', 'Veal', 'GameMeat', 'Pork']:
        return 'Meat'
    elif harmonize in ['Poultry', 'Duck', 'Chicken', 'Turkey']:
        return 'Poultry'
    elif harmonize in ['Shellfish']:
        return 'Shellfish'
    elif harmonize in ['Pasta']:
        return 'Pasta'
    elif harmonize in ['RichFish']:
        return 'RichFish'
    elif harmonize in ['Vegetarian','Mushrooms']:
        return 'Vegetarian'
    elif harmonize in ['Cheese', 'SoftCheese', 'MaturatedCheese', 'HardCheese', 'GoatCheese', 'BlueCheese']:
        return 'Cheese'
    elif harmonize in ['CuredMeat']:
        return 'CuredMeat'
    elif harmonize in ['SpicyFood']:
        return 'SpicyFood'
    elif harmonize in ['Appetizer', 'Snack']:
        return 'Appetizer and Snacks'
    elif harmonize in ['SweetDessert', 'FruitDessert', 'Dessert']:
        return 'Desserts'
    else:
        return harmonize

In [ ]:
def combine_type_classes(wine_type):
    if wine_type in ['Dessert','Dessert/Port']:
        return 'Dessert'
    else:
        return wine_type

In [ ]:
save_file_name = "Cleaned_Full_100K_wines_exploded"
wine_df_5 = pd.read_csv(f'~/code/ArjanAngenent/VinoDine/raw_data/{save_file_name}.csv')
mlb_harm = MultiLabelBinarizer(sparse_output=False)

wine_df_5.drop(columns=['Grapes'], inplace=True)
wine_df_5.Harmonize = wine_df_5.Harmonize.apply(lambda harmonize:combine_harmonize_classes(harmonize))
wine_df_5.Type = wine_df_5.Type.apply(lambda wine_type:combine_type_classes(wine_type))
wine_df_5.drop_duplicates(inplace=True)

wine_df_5_no_harm = wine_df_5.drop(columns='Harmonize').drop_duplicates()
wine_df_5_no_harm

wine_df_5_grouped = pd.DataFrame(wine_df_5.groupby(by='WineID')['Harmonize'].apply(list))
wine_df_5_merged = wine_df_5_grouped.join(wine_df_5.drop(columns='Harmonize')\
                                          .drop_duplicates().set_index('WineID'), on='WineID', how='left')

# Binary encode Harmonize(kinds of food)
wine_df_5_grouped_bin = wine_df_5_merged.join(pd.DataFrame(
    mlb_harm.fit_transform(element for element in wine_df_5_grouped.Harmonize),
    index=wine_df_5_grouped.index,
    columns=mlb_harm.classes_
    ))
wine_df_5_grouped_bin.drop(columns=['Harmonize'], inplace=True)
wine_df_5_grouped_bin_cleand = wine_df_5_grouped_bin.drop(columns=['WineName','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])


In [ ]:
X_5 = wine_df_5_grouped_bin_cleand[['Type','Body','Acidity', 'ABV']]
# X_3 = wine_df_2_bin_cleaned.iloc[:,:5]
y_5 = wine_df_5_grouped_bin_cleand.iloc[:,5:]

X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X_5, y_5, random_state=42, test_size=0.3)

X_train_5.shape, X_test_5.shape, y_train_5.shape, y_test_5.shape 

In [ ]:
plt.hist(X_train_5['Type'])

In [ ]:
# Define which columns need to be encoded
cat_cols = ['Type','Body','Acidity']
num_cols = ['ABV']
cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                        MinMaxScaler())
cat_num = make_pipeline(MinMaxScaler())

# Create preprocessor pipeline
preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

# Create binary classifier
binary_classifier = SGDClassifier(random_state=42)

ova_classifier = OneVsRestClassifier(binary_classifier)

pipeline = make_pipeline(preprocessing, ova_classifier)

In [ ]:
pipeline.fit(X_train_5, y_train_5)

y_pred_5 = ova_classifier.predict(preprocessing.transform(X_test_5))

# Evaluate performance
accuracy_5 = accuracy_score(y_test_5, y_pred_5)
print("Accuracy:", accuracy_5)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_5, y_pred_5))

In [ ]:
y_test_5.columns

# RandomOverSampling

In [ ]:
X_train_5.shape, X_test_5.shape, y_train_5.shape, y_test_5.shape 

In [ ]:
plt.hist(X_train['Type'])

In [ ]:
# features to select: Type, Elaborate, Grapes, ABV, Body, Acidity
# target: harmonize
y = wine_df_1[['Harmonize']]
X = wine_df_1[['Type','ABV', 'Body','Acidity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ros = RandomOverSampler(random_state=42)

X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

In [ ]:
pipeline.fit(X_train_res, y_train_res)

y_pred_res = ova_classifier.predict(preprocessing.transform(X_test))

# Evaluate performance
accuracy_res = accuracy_score(y_test, y_pred_res)
print("Accuracy:", accuracy_res)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_res))

# no grapes

In [ ]:
save_file_name = "Cleaned_Full_100K_wines"
mlb_harm = MultiLabelBinarizer(sparse_output=False)
mlb_grape = MultiLabelBinarizer(sparse_output=False)

wine_df_6 = pd.read_csv(f'~/code/ArjanAngenent/VinoDine/raw_data/{save_file_name}.csv')

wine_df_6.drop(columns=['Grapes'], inplace=True)

# Binary encode Harmonize(kinds of food)
wine_df_6_bin = wine_df_6.join(pd.DataFrame(
    mlb_harm.fit_transform(eval(element) for element in wine_df_6.Harmonize),
    index=wine_df_6.index,
    columns=mlb_harm.classes_
    ))
wine_df_6_bin.drop(columns=['Harmonize'], inplace=True)
wine_df_6_bin



# Create a list of the kind of grapes that are mentioned less then 2.000 times
harm_list = wine_df_6_bin.iloc[:,15:].sum() # sum the number of times a food is mentioned via column
harm_to_drop = harm_list[harm_list<=15_000].index.to_list() # create a list withe kind of food mentioned less then 50 times
wine_df_6_bin.drop(columns=harm_to_drop, inplace=True) # drop columns with food not mentioned more then 50 times
wine_df_6_bin = wine_df_6_bin[wine_df_6_bin.iloc[:,15:].eq(1).any(axis=1)] # drop wines which are not represented by a food anymore

#Drop addional columns not used for model
wine_df_6_bin_cleaned = wine_df_6_bin.drop(columns=['WineName', 'WineID','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])

wine_df_6_bin_cleaned

In [ ]:
harm_list

In [ ]:
harm_to_drop

In [ ]:
plt.figure(figsize=(15,15))
harm_list.plot(kind='bar')
plt.ylim(bottom=15_000);

In [ ]:
# Define which columns need to be encoded
cat_cols = make_column_selector(dtype_include='object')
num_cols = make_column_selector(dtype_include='number')
cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                        MinMaxScaler())
cat_num = MinMaxScaler()

# Create preprocessor pipeline
preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

# Create binary classifier
binary_classifier = SGDClassifier(max_iter=500, random_state=42)

ova_classifier = OneVsRestClassifier(binary_classifier)

pipeline = make_pipeline(preprocessing, ova_classifier)

In [ ]:
X_6 = wine_df_6_bin_cleaned[['Type','Elaborate','Body','Acidity', 'ABV']]
y_6 = wine_df_6_bin_cleaned.drop(columns=['Type','Elaborate','Body','Acidity', 'ABV'])

In [ ]:
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(X_6, y_6, test_size=0.3, random_state=42)

In [ ]:
pipeline.fit(X_train_6, y_train_6)

In [ ]:
y_pred_6 = ova_classifier.predict(preprocessing.transform(X_test_6))

In [ ]:
# Evaluate performance
accuracy_6 = accuracy_score(y_test_6, y_pred_6)
print("Accuracy:", accuracy_6)

In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(y_test_6, y_pred_6))